In [1]:
from dataset import CLIPDataModule
from termcolor import cprint
import pandas as pd
import time
import psutil
import matplotlib.pyplot as plt
from lidar_helper import get_affine_matrix_quat
%matplotlib inline

### Create Dataloader Using All Processed Pickle Files
batch size of 64 \
using the main process

In [2]:
num_workers = psutil.cpu_count()
print('available workers: ', num_workers)

available workers:  16


In [3]:
dm = CLIPDataModule(
    data_dir='./not_working/',
    batch_size=32,
    num_workers=16,
    joy_pred_len=240,
    include_lidar_file_names=True,
    visualize_goal=True
)

dm.setup()

  0%|                                                                                                                                                                                                                                                                                    | 0/1 [00:00<?, ?it/s]

Pickle file exists. Loading from pickle file


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.07s/it]

Delay frame is : 30


### Dataset

In [4]:
cprint(f"number of samples: {len(dm.dataset)}", 'green', attrs=['bold'])

number of samples: 2148


### Test Trainloader Iteration

<b>NOTE</b>: lidar batch data is returned as a tuple (lidar_stack, file_names) where file names is used for visualization/debug purposes

<b>Shape Information</b>: \
```lidar_stack.shape``` = (batch size, channels, img height, img width) \
```joystick.shape``` = (batch size, future cuttoff, xyz) \
```goal.shape``` = (batch size, 2)


In [5]:
# turn train_dataloader into iterable
trainloader = dm.train_dataloader()
train_iter = iter(trainloader)
len(trainloader)

50

In [6]:
%%time
for i, batch in enumerate(trainloader):
    lidar, joystick, goal = batch
    lidar_stack = lidar[0]
    lidar_fnames = lidar[1]
    if i == 1:
        print('lidar img batch shape: ', lidar_stack.shape)
        print('lidar file names batch shape: ', lidar_fnames.shape)
        print('joystick batch shape: ', joystick.shape)
        print('goal batch shape: ', goal.shape)

cprint(f'successfully retrieved all batches !!!', color='green', attrs=['bold'])

lidar img batch shape:  torch.Size([32, 5, 401, 401])
lidar file names batch shape:  torch.Size([32, 5])
joystick batch shape:  torch.Size([32, 240, 3])
goal batch shape:  torch.Size([32, 2])
successfully retrieved all batches !!!
CPU times: user 43 ms, sys: 503 ms, total: 546 ms
Wall time: 10 s


### Select a Random Sample from the Entire Dataset to Visualize

In [ ]:
import random

idx = random.randint(0, len(dm.dataset))

# get random sample
sample = dm.dataset[idx]

### Visualize Lidar Stack

Display 5 images from the randomly selected sample lidar stack

In [ ]:
from lidar_helper import visualize_lidar_stack

# visualize mode returns lidar stack
# and the corresponding file names
lidar_stack = sample[0][0]
lidar_fnames = sample[0][1]

cprint(f'index: {idx}, total offset: 50', color='green', attrs=['bold'])
visualize_lidar_stack(lidar_stack=lidar_stack, file_names=lidar_fnames)

### Visualize Future Joystick in Table View

In [ ]:
joystick = sample[1]
print('joystick dtype: ', joystick.dtype)
print('joystick shape: ', joystick.shape)

joy_df = pd.DataFrame(data=joystick, columns=['linear_x', 'linear_y', 'angular_z'])
joy_df

In [ ]:
joystick = sample[1]
print('joystick shape: ', joystick.shape)

joy_df = pd.DataFrame(data=joystick, columns=['linear_x', 'linear_y', 'angular_z'])
joy_df

### Display Some Joystick Stats

In [ ]:
linear_x_max = joy_df['linear_x'].max()
linear_x_min = joy_df['linear_y'].min()
linear_x_avg = joy_df['linear_x'].mean()

linear_y_max = joy_df['linear_y'].max()
linear_y_min = joy_df['linear_y'].min()
linear_y_avg = joy_df['linear_y'].mean()

angular_z_max = joy_df['angular_z'].max()
angular_z_min = joy_df['angular_z'].min()
angular_z_avg = joy_df['angular_z'].mean()

In [ ]:
joy_stats = {"linear_x": [linear_x_max, linear_x_min, linear_x_avg], "linear_y": [
    linear_y_max, linear_y_min, linear_y_avg], "angular_z": [angular_z_max, angular_z_min, angular_z_avg]}

joy_stats_df = pd.DataFrame(data=joy_stats, columns=[
                            'linear_x', 'linear_y', 'angular_z'], index=['max', 'min', 'avg'])

joy_stats_df


### Goal Information

In [ ]:
goal = sample[2]
print(f'x: {goal[0]:.2f}, y: {goal[1]:.2f}')